In [2]:
import os
import pandas as pd
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional
import concurrent.futures
from tqdm import tqdm
from time import sleep


from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
# Run chain
import warnings
warnings.filterwarnings('ignore')

In [3]:
load_dotenv()
token = os.environ["TOKEN2"]

In [4]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.lab45.ai/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)

       # print("API Response:", response.json())
        response.raise_for_status()

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [5]:
llm = LlamaLLM()

In [6]:
ROLE_CLUSTER_MASTER = pd.read_csv(r"D:\OneDrive - Wipro\Desktop\Static Dumps\Role-Skill-Cluster Master Report.csv")

In [7]:
ROLE_CLUSTER_MASTER_REQD=ROLE_CLUSTER_MASTER[['CLUSTER_NAME','SKILL_NAME']]
ROLE_CLUSTER_MASTER_REQD

,CLUSTER_NAME,SKILL_NAME
0,SENIOR CONSULTANT - TECH AND IMPL CONSUMER GOO...,"SSIS (L2), Application Testing (L2), Business ..."
1,LEAD ADMINISTRATOR VMWARE AWS CLOUD - L1,"VMware Server Admin (L3), Windows and Vmware I..."
2,Tools Engineer ASSET MANAGEMENT -SNOW - L3,"ITIL Asset-Config Mgmt (L1), Software Asset Ma..."
3,DOMAIN CONSULTANT CORE BANKING - L2,"Core Banking (L2), Payments (L1)"
4,TEST LEAD TEST DATA MANAGEMENT - L1,"Mainframe Testing (L1), Python Scripting (L1),..."
...,...,...
8039,Solution Architect Enterprise Asset Management...,"Solution Architecture (L1), IBM Maximo Asset M..."
8040,Solution Architect BSM AUTOMATION - BMC - L1,"Consulting (L2), VB Scripting (L2), BMC BladeL..."
8041,Functional Consultant SAP Source To Pay Ariba ...,"Ariba Upstream (L3), Ariba Downstream (L3), SA..."
8042,TECHNO FUNCTIONAL CONSULTANT PRIVILEGE ACCESS ...,"Consulting (L2), BeyondTrust (L3)"


In [8]:
# JD = []
# y = 0
# z = 3
# for i, row in enumerate(ROLE_CLUSTER_MASTER_REQD.itertuples()):
#   if y <= i <= z:  
#     print(i)
#     skills=row.SKILL_NAME
#     title=row.CLUSTER_NAME
#     company="Wipro Ltd."
#     user = "user"
#     prompt = """ As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to meticulously evaluate skills and title provided and give detailed job description having below points outlined.
#                 Use the mentioned skills along with proficiency  & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
#                 Job Role: 
#                 Give skill specified by user
#                 Clearly state the role/title of the position and give brief summary of the role/title in 3-4 lines

#                 Key Responsibilities:
#                 Title/Role will have overall impact on responsibilities
#                 Utilize expertise in [Key Skill 1] and [title/Role] to [Responsibility 1] .
#                 Apply advanced knowledge of [Key Skill 2]  to [Responsibility 2].
#                 Demonstrate proficiency in [Key Skill 3] to [Responsibility 3].
#                 Utilize [Key Skill 4]  to [Responsibility 4].
#                 Apply [Key Skill 5] to [Responsibility 5].

#                 Qualifications and Skills:

#                 Proficiency in [Key Skill 1] is essential for this role, with a strong focus on [Specific Requirement].
#                 Advanced knowledge of [Key Skill 2] is highly desirable, with the ability to [Specific Requirement].
#                 Familiarity with [Key Skill 3] is beneficial, with the capacity to [Specific Requirement].
#                 Experience in [Key Skill 4] is advantageous, with the ability to [Specific Requirement].
#                 Proficiency in [Key Skill 5] is preferred, with the capability to [Specific Requirement].

#                 Experience Requirements:

#                 A minimum of [Number of Years] years of relevant experience in [Industry/Field].
#                 Previous experience in [Specific Type of Experience] is beneficial, with a focus on [Relevant Proficiency].
#                 Demonstrated proficiency in [Key Skill 1] and [Key Skill 2] through [Type of Experience].
#                 Proven track record of applying [Key Skill 3] and [Key Skill 4] in [Type of Experience].

#                 """ + skills + title + company # type: ignore
#     result = llm._call(prompt,user)
#     parsed_result = result['data']['content'] # type: ignore
#     JD.append("CLUSTER NAME :"+title +'\n'+"CLUSTER SKILLS : "+skills+'\n'+parsed_result+'\n'+'\n'+'\n'+'\n') # type: ignore


In [9]:
temprompt = """ As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to meticulously evaluate skills and title provided and give detailed job description having below points outlined.
                Use the mentioned skills along with proficiency  & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
                Job Role: 
                Give skill specified by user
                Clearly state the role/title of the position and give brief summary of the role/title in 3-4 lines

                Key Responsibilities:
                Title/Role will have overall impact on responsibilities
                Utilize expertise in [Key Skill 1] and [title/Role] to [Responsibility 1] .
                Apply advanced knowledge of [Key Skill 2]  to [Responsibility 2].
                Demonstrate proficiency in [Key Skill 3] to [Responsibility 3].
                Utilize [Key Skill 4]  to [Responsibility 4].
                Apply [Key Skill 5] to [Responsibility 5].

                Qualifications and Skills:

                Proficiency in [Key Skill 1] is essential for this role, with a strong focus on [Specific Requirement].
                Advanced knowledge of [Key Skill 2] is highly desirable, with the ability to [Specific Requirement].
                Familiarity with [Key Skill 3] is beneficial, with the capacity to [Specific Requirement].
                Experience in [Key Skill 4] is advantageous, with the ability to [Specific Requirement].
                Proficiency in [Key Skill 5] is preferred, with the capability to [Specific Requirement].

                Experience Requirements:

                A minimum of [Number of Years] years of relevant experience in [Industry/Field].
                Previous experience in [Specific Type of Experience] is beneficial, with a focus on [Relevant Proficiency].
                Demonstrated proficiency in [Key Skill 1] and [Key Skill 2] through [Type of Experience].
                Proven track record of applying [Key Skill 3] and [Key Skill 4] in [Type of Experience]. """

In [10]:
def generate_jds(temprompt, start_index=0, end_index=1000):
    """
    Generate Job Descriptions for a specific range of roles
    
    Parameters:
    start_index (int): Starting index for processing
    end_index (int): Ending index for processing
    """
    JD = []
    failed = []
    

    # Validate indices
    if start_index < 0:
        start_index = 0
    if end_index > len(ROLE_CLUSTER_MASTER_REQD):
        end_index = len(ROLE_CLUSTER_MASTER_REQD)
    
    try:
        for i, row in enumerate(ROLE_CLUSTER_MASTER_REQD.itertuples()):
            if start_index <= i <= end_index:
                print(f"Processing record {i} of {end_index}")
                #print(row)
                skills = row.SKILL_NAME
                title = row.CLUSTER_NAME
                company = "Wipro Ltd"
                user = "user"
                
                prompt = temprompt + skills + title + company # type: ignore
                
                try:
                    result = llm._call(prompt, user)
                    parsed_result = result['data']['content'] # type: ignore
                    jd_entry = (
                        f"CLUSTER NAME: {title}\n"
                        f"CLUSTER SKILLS: {skills}\n"
                        f"{parsed_result}\n\n\n\n"
                    )
                    JD.append(jd_entry)
                except Exception as ce:
                    print(f"Connection error for index {i}: {ce}")
                    print(f"Re-processing record {i}")
                    result = llm._call(prompt, user)
                    parsed_result = result['data']['content'] # type: ignore
                    jd_entry = (
                        f"CLUSTER NAME: {title}\n"
                        f"CLUSTER SKILLS: {skills}\n"
                        f"{parsed_result}\n\n\n\n"
                    )
                    JD.append(jd_entry)
                    continue
                    
    except Exception as e:
        print(f"Error processing record {i}: {str(e)}")
        failed.append(i)
        sleep(15)
        print(f"Re-processing record {i}")
        result = llm._call(prompt, user)
        parsed_result = result['data']['content'] # type: ignore
        jd_entry = (
                        f"CLUSTER NAME: {title}\n"
                        f"CLUSTER SKILLS: {skills}\n"
                        f"{parsed_result}\n\n\n\n"
                    )
        JD.append(jd_entry)  
    
    
    return JD

In [ ]:
# Example usage:
# Generate JDs for n records
jd_results = generate_jds(temprompt,5001, 8000)

Processing record 5001 of 8000
Processing record 5002 of 8000
Processing record 5003 of 8000
Processing record 5004 of 8000
Processing record 5005 of 8000
Processing record 5006 of 8000
Processing record 5007 of 8000
Processing record 5008 of 8000
Processing record 5009 of 8000
Processing record 5010 of 8000
Processing record 5011 of 8000
Processing record 5012 of 8000
Processing record 5013 of 8000
Processing record 5014 of 8000
Processing record 5015 of 8000
Processing record 5016 of 8000
Processing record 5017 of 8000
Processing record 5018 of 8000
Processing record 5019 of 8000
Connection error for index 5019: HTTPSConnectionPool(host='api.lab45.ai', port=443): Max retries exceeded with url: /v1.1/skills/completion/query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000251D5ABC110>: Failed to resolve 'api.lab45.ai' ([Errno 11001] getaddrinfo failed)"))
Re-processing record 5019
Processing record 5020 of 8000
Processing record 5021 of 8000
Process

In [12]:
# Save results to file
with open('outfilenew', 'a') as f:
    for jd in jd_results:
        f.write(jd)
print("Writing to file complete....")

Writing to file complete....


In [13]:
# with open("outfilenew2", "w") as outfile:
#     outfile.write("\n".join(JD))